In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.autograd import grad
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

print(torch.__version__)

2.9.1


In [2]:
print(torch.cuda.is_available())

False


In [3]:
print("CUDA available:", torch.cuda.is_available())
print("MPS available:", getattr(torch.backends, "mps", None) is not None and torch.backends.mps.is_available())

device = torch.device("mps")
print("Using device:", device)

CUDA available: False
MPS available: True
Using device: mps


In [4]:
tensor0d = torch.tensor(1)

tensor1d = torch.tensor([1, 2, 3])

tensor2d = torch.tensor([
    [1, 2],
    [3, 4]
])

tensor3d_1 = torch.tensor([
    [[1, 2], [3, 4]],
    [[5, 6], [7, 8]]
])

ary3d = np.array([[[1, 2], [3, 4]],
                  [[5, 6], [7, 8]]])
tensor3d_2 = torch.tensor(ary3d)  # Copies NumPy array
tensor3d_3 = torch.from_numpy(ary3d)  # Shares memory with NumPy array

In [5]:
ary3d[0, 0, 0] = 999
print(tensor3d_2)
print(tensor3d_3)

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])
tensor([[[999,   2],
         [  3,   4]],

        [[  5,   6],
         [  7,   8]]])


In [6]:
y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2])
b = torch.tensor([0.0])

z = w1 * x1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)
print(loss)

tensor(0.0852)


In [8]:
y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [9]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs: int, num_outputs: int):
        super().__init__()

        self.layers = torch.nn.Sequential(

            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logits = self.layers(x)
        return logits

In [10]:
model = NeuralNetwork(num_inputs=50, num_outputs=3)

In [11]:
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [12]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [13]:
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.1207,  0.0194, -0.0372,  ..., -0.1248, -0.1029, -0.0369],
        [ 0.0530, -0.0645,  0.1330,  ..., -0.0449, -0.0499,  0.1301],
        [ 0.1200,  0.0654, -0.1226,  ..., -0.0758,  0.0324,  0.0775],
        ...,
        [-0.0234,  0.0414,  0.0565,  ..., -0.0618, -0.0081,  0.0862],
        [ 0.0779,  0.0014, -0.1405,  ..., -0.0008, -0.1242,  0.0202],
        [ 0.0971,  0.0806, -0.0424,  ...,  0.0221,  0.0156,  0.1402]],
       requires_grad=True)


In [14]:
print(model.layers[0].weight.shape)
print(model.layers[0].bias.shape)

torch.Size([30, 50])
torch.Size([30])


In [15]:
torch.manual_seed(123)

model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [18]:
x_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

x_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [19]:
class ToyDataset(Dataset):
    def __init__(self, x: torch.Tensor, y: torch.Tensor):
        self.features = x
        self.lables = y

    def __getitem__(self, index: int):
        return self.features[index], self.lables[index]

    def __len__(self) -> int:
        return self.lables.shape[0]

train_ds = ToyDataset(x_train, y_train)
test_ds = ToyDataset(x_test, y_test)

In [20]:
len(train_ds)

5

In [21]:
torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [22]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx}:", x, y)

Batch 0: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [24]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):

        logits = model(features)

        loss = F.cross_entropy(logits, labels) # Loss function

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")

    model.eval()

Epoch: 001/003 | Batch 000/003 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/003 | Train/Val Loss: 0.65
Epoch: 001/003 | Batch 002/003 | Train/Val Loss: 0.42
Epoch: 002/003 | Batch 000/003 | Train/Val Loss: 0.05
Epoch: 002/003 | Batch 001/003 | Train/Val Loss: 0.13
Epoch: 002/003 | Batch 002/003 | Train/Val Loss: 0.00
Epoch: 003/003 | Batch 000/003 | Train/Val Loss: 0.01
Epoch: 003/003 | Batch 001/003 | Train/Val Loss: 0.00
Epoch: 003/003 | Batch 002/003 | Train/Val Loss: 0.02


In [25]:
model.eval()

with torch.no_grad():
    outputs = model(X_train)

print(outputs)

tensor([[ 2.9320, -4.2563],
        [ 2.6045, -3.8389],
        [ 2.1484, -3.2514],
        [-2.1461,  2.1496],
        [-2.5004,  2.5210]])
